In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#df_pl = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20_pl_g.csv')
df_pl = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_01/Stats/df_all_round_sim.csv')

In [2]:
df_pl = df_pl.sort_values(by=['start_date','match_id','innings','ball']).reset_index(drop=True)

In [3]:
df_pl['phase'] = np.where(df_pl['ball']<37, 'pp', 
                        np.where(df_pl['ball']>90, 'death',
                            'middle'))

df_pl[df_pl.striker=='V Kohli'].shape

In [4]:
player_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='season_02')

players_list = player_list['Func_Name'].dropna().tolist()

#players_list = players_list.remove('nan')

print(type(players_list))

len(players_list)

<class 'list'>


241

In [5]:
bat_pl = df_pl[df_pl.striker.isin(players_list)]
bowl_pl = df_pl[df_pl.bowler.isin(players_list)]

bat_pl.shape, bowl_pl.shape

((17178, 54), (16772, 54))

In [6]:
bat_pl_1 = bat_pl[bat_pl.innings==1]
bat_pl_2 = bat_pl[bat_pl.innings==2]

bowl_pl_1 = bowl_pl[bowl_pl.innings==1]
bowl_pl_2 = bowl_pl[bowl_pl.innings==2]


# Initialize dictionaries to store match IDs
batter_match_id_1 = {}
batter_match_id_2 = {}
bowler_match_id_1 = {}
bowler_match_id_2 = {}

# Collect last 5 match IDs for each batter in bat_pl_1
for batter in bat_pl_1.striker.unique():
    matches = bat_pl_1[bat_pl_1.striker == batter].match_id.unique()[-5:].tolist()
    batter_match_id_1[batter] = matches

# Collect last 5 match IDs for each batter in bat_pl_2
for batter in bat_pl_2.striker.unique():
    matches = bat_pl_2[bat_pl_2.striker == batter].match_id.unique()[-5:].tolist()
    batter_match_id_2[batter] = matches

# Collect last 5 match IDs for each bowler in bowl_pl_1
for bowler in bowl_pl_1.bowler.unique():
    matches = bowl_pl_1[bowl_pl_1.bowler == bowler].match_id.unique()[-5:].tolist()
    bowler_match_id_1[bowler] = matches

# Collect last 5 match IDs for each bowler in bowl_pl_2
for bowler in bowl_pl_2.bowler.unique():
    matches = bowl_pl_2[bowl_pl_2.bowler == bowler].match_id.unique()[-5:].tolist()
    bowler_match_id_2[bowler] = matches


In [7]:
# odd indices == player name
# even indices == match_id array

In [8]:
len(bowler_match_id_2), type(bowler_match_id_2)

(87, dict)

In [9]:
# Accessing the last 5 matches for a specific batter
bowler_name = 'T Natarajan'
last_5_matches = bowler_match_id_2.get(bowler_name, [1]) 

last_5_matches

['M034', 'M041', 'M046', 'M053', 'M060']

In [10]:
df_bat_1 = pd.DataFrame()

for batter_name in players_list:
    last_5_matches = batter_match_id_1.get(batter_name, [1])
    df_sub = bat_pl_1[(bat_pl_1.striker==batter_name)&(bat_pl_1.match_id.isin(last_5_matches))]
    if df_sub.shape[0]>0:
        df_sub.reset_index(drop=True, inplace=True)
        df_bat_1 = df_bat_1.append(df_sub)
    else:
        continue
    
    
df_bat_1.shape    

(6484, 54)

In [11]:
df_bat_2 = pd.DataFrame()

for batter_name in players_list:
    last_5_matches = batter_match_id_2.get(batter_name, [2])
    df_sub = bat_pl_2[(bat_pl_2.striker==batter_name)&(bat_pl_2.match_id.isin(last_5_matches))]
    if df_sub.shape[0]>0:
        df_sub.reset_index(drop=True, inplace=True)
        df_bat_2 = df_bat_2.append(df_sub)
    else:
        continue
    
    
df_bat_2.shape    

(6499, 54)

In [12]:
df_bowl_1 = pd.DataFrame()

for bowler_name in players_list:
    last_5_matches = bowler_match_id_1.get(bowler_name, [1])
    df_sub = bowl_pl_1[(bowl_pl_1.bowler==bowler_name)&(bowl_pl_1.match_id.isin(last_5_matches))]
    if df_sub.shape[0]>0:
        df_sub.reset_index(drop=True, inplace=True)
        df_bowl_1 = df_bowl_1.append(df_sub)
    else:
        continue
    
    
df_bowl_1.shape    

(7015, 54)

In [13]:
df_bowl_2 = pd.DataFrame()

for bowler_name in players_list:
    last_5_matches = bowler_match_id_2.get(bowler_name, [2])
    df_sub = bowl_pl_2[(bowl_pl_2.bowler==bowler_name)&(bowl_pl_2.match_id.isin(last_5_matches))]
    if df_sub.shape[0]>0:
        df_sub.reset_index(drop=True, inplace=True)
        df_bowl_2 = df_bowl_2.append(df_sub)
    else:
        continue
    
    
df_bowl_2.shape    

(6658, 54)

In [14]:
df_bowl_1['runs_conceeded'] = df_bowl_1['runs_off_bat']+df_bowl_1['wides']+df_bowl_1['noballs']
df_bowl_2['runs_conceeded'] = df_bowl_2['runs_off_bat']+df_bowl_2['wides']+df_bowl_2['noballs']

In [15]:
## bat,bowl stats, innings wise

In [16]:
batter_stats_1_form = df_bat_1.groupby('striker').agg(

num_innings = ('match_id','nunique'),
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter' ,'sum'),
    outs = ('is_striker_Out','sum'),
    bat_order = ('striker_batting_position', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    
).reset_index()

batter_stats_2_form = df_bat_2.groupby('striker').agg(

num_innings = ('match_id','nunique'),
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter' ,'sum'),
    outs = ('is_striker_Out','sum'),
    bat_order = ('striker_batting_position', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    
).reset_index()

bowler_stats_1_form = df_bowl_1.groupby('bowler').agg(

num_innings = ('match_id','nunique'),
    runs = ('runs_conceeded','sum'),
    balls = ('islegal' ,'sum'),
    wkts = ('isBowlerWicket','sum'),
    bowl_phase = ('phase', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    
).reset_index()

bowler_stats_2_form = df_bowl_2.groupby('bowler').agg(

num_innings = ('match_id','nunique'),
    runs = ('runs_conceeded','sum'),
    balls = ('islegal' ,'sum'),
    wkts = ('isBowlerWicket','sum'),
    bowl_phase = ('phase', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    
).reset_index()

batter_stats_1_form['bat_avg'] = batter_stats_1_form['runs']/batter_stats_1_form['outs']
batter_stats_2_form['bat_avg'] = batter_stats_2_form['runs']/batter_stats_2_form['outs']

batter_stats_1_form['bat_sr'] = 100*batter_stats_1_form['runs']/batter_stats_1_form['balls']
batter_stats_2_form['bat_sr'] = 100*batter_stats_2_form['runs']/batter_stats_2_form['balls']

bowler_stats_1_form['bowl_sr'] = bowler_stats_1_form['balls']/bowler_stats_1_form['wkts']
bowler_stats_2_form['bowl_sr'] = bowler_stats_2_form['balls']/bowler_stats_2_form['wkts']

bowler_stats_1_form['bowl_eco'] = 6*bowler_stats_1_form['runs']/bowler_stats_1_form['balls']
bowler_stats_2_form['bowl_eco'] = 6*bowler_stats_2_form['runs']/bowler_stats_2_form['balls']

In [17]:
##example
#batter_stats_1_form[batter_stats_1_form.striker=='Avesh Khan']
bowler_stats_1_form[bowler_stats_1_form.bowler=='Avesh Khan']

,bowler,num_innings,runs,balls,wkts,bowl_phase,bowl_sr,bowl_eco
9,Avesh Khan,5,167,114,4,pp,28.5,8.789474


In [18]:
batter_stats_1_form[batter_stats_1_form.bat_sr<50]

,striker,num_innings,runs,balls,outs,bat_order,bat_avg,bat_sr
19,CV Varun,1,1,3,0,11,inf,33.333333
46,JP Inglis,1,0,1,1,4,0.0,0.000000
53,Kuldeep Yadav,2,1,6,1,10,1.0,16.666667
68,MP Yadav,1,0,2,1,8,0.0,0.000000
81,Noor Ahmad,1,0,1,1,9,0.0,0.000000
91,R Powell,1,4,9,1,6,4.0,44.444444


In [19]:
## forming the form-factor

## bat form factors

for batters:
things to consider -- bat_order, bat_avg, bat_sr, balls

1. bat_order & bat_avg will cancel themselves. (i.e., if avg is up, but bat_order is down, then bat_order should
reduce the effect of avg.

a=(bat_avg**0.2)*(bat_order**-0.2)

2. bat_sr & bat_order will cancel themselves. (i.e., if sr is up, but bat_order is down, then bat_order should
reduce the effect of sr.

b=(bat_sr/100**0.2)*(bat_order**-0.2)

3. balls & bat_order will help themselves.

c=(balls*bat_order)**0.01


c*(a+b)*0.5

In [20]:
batter_stats_1_form['bat_avg'] = batter_stats_1_form.apply(
    lambda row: row['runs'] if np.isinf(row['bat_avg']) else row['bat_avg'], axis=1
)

batter_stats_1_form['bat_avg'] = batter_stats_1_form['bat_avg'].fillna(0)

batter_stats_2_form['bat_avg'] = batter_stats_2_form.apply(
    lambda row: row['runs'] if np.isinf(row['bat_avg']) else row['bat_avg'], axis=1
)

batter_stats_2_form['bat_avg'] = batter_stats_2_form['bat_avg'].fillna(0)

mult_1 = 0.2
mult_2 = 0.01
mult_3 = 0.3

batter_stats_1_form['a'] = ((0.5*batter_stats_1_form['bat_avg'])**mult_1)*((batter_stats_1_form['bat_order']**0.5)**-mult_1)
batter_stats_1_form['b'] = ((batter_stats_1_form['bat_sr']/100)**mult_3)*((batter_stats_1_form['bat_order']**0.5)**-mult_1)
batter_stats_1_form['c'] = (batter_stats_1_form['balls']*(batter_stats_1_form['bat_order']**0.5))**mult_2


batter_stats_1_form['form'] = 0.5*(batter_stats_1_form['a']+batter_stats_1_form['b'])*batter_stats_1_form['c']

batter_stats_1_form['form'] = batter_stats_1_form['form']/batter_stats_1_form['form'].mean()

batter_stats_1_form['form'] = batter_stats_1_form['form']**0.75

batter_stats_1_form['form'] = batter_stats_1_form['form']/batter_stats_1_form['form'].mean()


batter_stats_2_form['a'] = ((0.5*batter_stats_2_form['bat_avg'])**mult_1)*((batter_stats_2_form['bat_order']**0.5)**-mult_1)
batter_stats_2_form['b'] = ((batter_stats_2_form['bat_sr']/200)**mult_3)*((batter_stats_2_form['bat_order']**0.5)**-mult_1)
batter_stats_2_form['c'] = (batter_stats_2_form['balls']*(batter_stats_2_form['bat_order']**0.5))**mult_2


batter_stats_2_form['form'] = 0.5*(batter_stats_2_form['a']+batter_stats_2_form['b'])*batter_stats_2_form['c']

batter_stats_2_form['form'] = batter_stats_2_form['form']/batter_stats_2_form['form'].mean()

batter_stats_2_form['form'] = batter_stats_2_form['form']**0.75

batter_stats_2_form['form'] = batter_stats_2_form['form']/batter_stats_2_form['form'].mean()


In [21]:
batter_stats_1_form.form.describe()

count    139.000000
mean       1.000000
std        0.204134
min        0.000000
25%        0.930469
50%        1.029089
75%        1.122616
max        1.328012
Name: form, dtype: float64

In [22]:
batter_stats_1_form[batter_stats_1_form.form>1.25].sort_values(by='form')

,striker,num_innings,runs,balls,outs,bat_order,bat_avg,bat_sr,a,b,c,form
7,Abhishek Sharma,5,190,103,5,1,38.00,184.466019,1.801983,1.201641,1.047438,1.260503
97,RD Gaikwad,5,191,119,4,1,47.75,160.504202,1.886204,1.152513,1.048952,1.272910
77,N Sindhu,2,126,55,2,2,63.00,229.090909,1.860198,1.196463,1.044501,1.274472
31,GJ Maxwell,5,203,126,4,1,50.75,161.111111,1.909331,1.153819,1.049551,1.281127
126,TM Head,5,220,146,4,1,55.00,150.684932,1.940290,1.130892,1.051099,1.285065
119,Shubman Gill,4,228,148,3,1,76.00,154.054054,2.069935,1.138418,1.051242,1.328012


bowler_stats_1_form[bowler_stats_1_form.form>1.35].sort_values(by='form')

In [23]:
batter_stats_1_form['form'] = np.where(batter_stats_1_form.balls<5, 0, batter_stats_1_form['form'])
batter_stats_2_form['form'] = np.where(batter_stats_2_form.balls<5, 0, batter_stats_2_form['form'])


In [24]:
##example
batter_stats_1_form[batter_stats_1_form.striker=='KL Rahul']
#bowler_stats_1_form[bowler_stats_1_form.bowler=='Avesh Khan']

,striker,num_innings,runs,balls,outs,bat_order,bat_avg,bat_sr,a,b,c,form
52,KL Rahul,5,162,110,5,1,32.4,147.272727,1.745432,1.123147,1.048127,1.218355


## bowl form factors

for bowlers:
things to consider -- bowl_phase, bowl_eco, bowl_sr, balls

1. bowl_phase & bowl_eco will cancel themselves. (i.e., if eco is up, but phase is death, then phase should
reduce the effect of eco.

a=(bowl_eco**1.05)if phase==middle,(bowl_eco**0.95)if phase==death, (bowl_eco**0.99)if phase==powerplay 

2. bowl_sr & bowl_phase will cancel themselves. (i.e., if sr is down, but phase is death, then phase should
reduce the effect of sr.

b=(bowl_sr)if phase==middle,(bowl_sr**1.05)if phase==death, (bowl_sr**0.95)if phase==powerplay 


In [25]:
bowler_stats_1_form['bowl_eco'] = bowler_stats_1_form.apply(
    lambda row: row['runs'] if np.isinf(row['bowl_eco']) else row['bowl_eco'], axis=1
)

bowler_stats_1_form['bowl_eco'] = bowler_stats_1_form['bowl_eco'].fillna(0)

bowler_stats_2_form['bowl_eco'] = bowler_stats_2_form.apply(
    lambda row: row['runs'] if np.isinf(row['bowl_eco']) else row['bowl_eco'], axis=1
)

bowler_stats_2_form['bowl_eco'] = bowler_stats_2_form['bowl_eco'].fillna(0)

bowler_stats_1_form['bowl_sr'] = bowler_stats_1_form.apply(
    lambda row: row['runs'] if np.isinf(row['bowl_sr']) else row['bowl_sr'], axis=1
)

#bowler_stats_1_form['bowl_sr'] = bowler_stats_1_form['bowl_eco'].fillna(0)

bowler_stats_2_form['bowl_sr'] = bowler_stats_2_form.apply(
    lambda row: row['runs'] if np.isinf(row['bowl_sr']) else row['bowl_sr'], axis=1
)

In [26]:
bowler_stats_1_form['a'] = np.where(bowler_stats_1_form['bowl_phase']=='middle', bowler_stats_1_form['bowl_eco']*1.05,
                                (np.where(bowler_stats_1_form['bowl_phase']=='death', bowler_stats_1_form['bowl_eco']*0.95,
                                        bowler_stats_1_form['bowl_eco']*0.99)))

bowler_stats_1_form['b'] = np.where(bowler_stats_1_form['bowl_phase']=='middle', bowler_stats_1_form['bowl_sr'],
                                (np.where(bowler_stats_1_form['bowl_phase']=='death', bowler_stats_1_form['bowl_sr']*0.95,
                                        bowler_stats_1_form['bowl_sr']*0.95)))

bowler_stats_1_form['c'] = np.where(bowler_stats_1_form['wkts']>0,bowler_stats_1_form['wkts']/15,1/15)

bowler_stats_1_form['d'] = (1/(bowler_stats_1_form['a']))*(1/(bowler_stats_1_form['b']))*\
                                                    (bowler_stats_1_form['c'])


bowler_stats_1_form['form'] = bowler_stats_1_form['d']**0.1
#**bowler_stats_1_form['d']

bowler_stats_1_form['form'] = bowler_stats_1_form['form']/bowler_stats_1_form['form'].mean()
bowler_stats_1_form['form'] = np.where(bowler_stats_1_form.balls<12, 0, bowler_stats_1_form['form'])

In [27]:
bowler_stats_1_form.form.describe()

count    97.000000
mean      0.949328
std       0.260024
min       0.000000
25%       0.876493
50%       0.981786
75%       1.077163
max       1.324904
Name: form, dtype: float64

In [28]:
bowler_stats_1_form[bowler_stats_1_form.form>1].sort_values(by='form')

,bowler,num_innings,runs,balls,wkts,bowl_phase,bowl_sr,bowl_eco,a,b,c,d,form
46,MJ Suthar,1,47,24,2,middle,12.000000,11.750000,12.337500,12.000000,0.133333,0.000901,1.001937
70,Ramandeep Singh,3,43,24,2,middle,12.000000,10.750000,11.287500,12.000000,0.133333,0.000984,1.010889
41,M Prasidh Krishna,5,169,120,4,middle,30.000000,8.450000,8.872500,30.000000,0.266667,0.001002,1.012669
43,M Theekshana,5,160,115,4,middle,28.750000,8.347826,8.765217,28.750000,0.266667,0.001058,1.018226
90,Tanush Kotian,2,89,48,3,middle,16.000000,11.125000,11.681250,16.000000,0.200000,0.001070,1.019364
57,NT Ellis,5,175,114,4,pp,28.500000,9.210526,9.118421,27.075000,0.266667,0.001080,1.020318
52,Mohammed Siraj,4,172,96,4,pp,24.000000,10.750000,10.642500,22.800000,0.266667,0.001099,1.022084
9,Avesh Khan,5,167,114,4,pp,28.500000,8.789474,8.701579,27.075000,0.266667,0.001132,1.025103
39,M Markande,5,147,108,4,middle,27.000000,8.166667,8.575000,27.000000,0.266667,0.001152,1.026891
95,YS Chahal,4,81,66,3,middle,22.000000,7.363636,7.731818,22.000000,0.200000,0.001176,1.029011


In [ ]:
bowler_stats_2_form['a'] = np.where(bowler_stats_2_form['bowl_phase']=='middle', bowler_stats_2_form['bowl_eco']*1.05,
                                (np.where(bowler_stats_2_form['bowl_phase']=='death', bowler_stats_2_form['bowl_eco']*0.95,
                                        bowler_stats_2_form['bowl_eco']*0.99)))

bowler_stats_2_form['b'] = np.where(bowler_stats_2_form['bowl_phase']=='middle', bowler_stats_2_form['bowl_sr'],
                                (np.where(bowler_stats_2_form['bowl_phase']=='death', bowler_stats_2_form['bowl_sr']*0.95,
                                        bowler_stats_2_form['bowl_sr']*0.95)))

bowler_stats_2_form['c'] = np.where(bowler_stats_2_form['wkts']>0,bowler_stats_2_form['wkts']/15,1/15)

bowler_stats_2_form['d'] = (1/(bowler_stats_2_form['a']))*(1/(bowler_stats_2_form['b']))*\
                                                    (bowler_stats_2_form['c'])


bowler_stats_2_form['form'] = bowler_stats_2_form['d']**0.1
#**bowler_stats_2_form['d']

bowler_stats_2_form['form'] = bowler_stats_2_form['form']/bowler_stats_2_form['form'].mean()
bowler_stats_2_form['form'] = np.where(bowler_stats_2_form.balls<12, 0, bowler_stats_2_form['form'])

In [ ]:
bowler_stats_2_form.form.describe()

In [31]:
bowler_stats_2_form[bowler_stats_2_form.form>1.5].sort_values(by='form', ascending=False)

,bowler,num_innings,runs,balls,wkts,bowl_phase,bowl_sr,bowl_eco,a,b,c,d,form


In [ ]:
##example
batter_stats_1_form[batter_stats_1_form.striker=='A Badoni']
#bowler_stats_1_form[bowler_stats_1_form.bowler=='Avesh Khan']

In [ ]:
batter_stats_1_form_trimmed = batter_stats_1_form[batter_stats_1_form.balls>=5].drop(['a','b','c'],axis=1)
batter_stats_2_form_trimmed = batter_stats_2_form[batter_stats_2_form.balls>=5].drop(['a','b','c'],axis=1)

bowler_stats_1_form_trimmed = bowler_stats_1_form[bowler_stats_1_form.balls>=12].drop(['a','b','c','d'],axis=1)
bowler_stats_2_form_trimmed = bowler_stats_2_form[bowler_stats_2_form.balls>=12].drop(['a','b','c','d'],axis=1)


In [ ]:
batter_stats_1_form_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn1.csv',
                           index=None)
batter_stats_2_form_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn2.csv',
                           index=None)

bowler_stats_1_form_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn1.csv',
                           index=None)
bowler_stats_2_form_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn2.csv',
                           index=None)